In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import numpy as np
import scipy.sparse as sp
import networkx as nx
import matplotlib.pyplot as plt
import scipy.optimize
import collections
import math
import time
import itertools
from scipy.linalg import expm, sinm, cosm, logm

In [ ]:
N=2
sx = sp.csr_matrix([[0.0,1.0],[1.0,0.0]])
sz = sp.csr_matrix([[1.0,0.0],[0.0,-1.0]])
sy = -1j * (sz @ sx)

def generate_operators(op, N):
    return [sp.kron(sp.eye(2**i),
                    sp.kron(op, sp.eye(2**(N-i-1))))
            for i in range(N)]

sx = generate_operators(sx, N)
sy = generate_operators(sy, N)
sz = generate_operators(sz, N)
p= np.random.normal(0,3,int(N*(N-1)/2+2*N))

        
                
def cost(args):
        
    Psi=np.ones(2**N)*1/(2**(N/2))     
    Index=np.zeros(N)
    Index[0:2]=3
    a=list(itertools.permutations(Index))
    a=list(dict.fromkeys(a))
    H=0
    for i in range(int(N*(N-1)/2)):
            
        s=1
        for j in a[i]:
                
            if j==0:
                s1=sp.csr_matrix([[1.0,0.0],[0.0,1.0]])
            if j==3:
                s1=sp.csr_matrix([[1.0,0.0],[0.0,-1.0]])                    
            s=sp.kron(s,s1)
                
        H=H+args[i]*s
        
    for j in range(int(N*(N-1)/2)):
            
        H=H+sz[j]*args[j+int(N*(N-1)/2)]+sx[j]*args[j+int(N*(N-1)/2+N)]
        
    Yexp=np.conj(expm(H)@Psi)*(expm(H)@Psi)
    Yteo=[0.1,0.4,0.3,0.2]
    cost=0
    for i in range(2**N):
            
        cost=Yteo[i]*np.log(Yexp[i])
             
    return cost
             
opt = scipy.optimize.minimize(cost, p)
opt['x']
        